# Google Research Football

Exploration of *Rule Based and Reinforcement Learning Hybrid solution* for the Kaggle Competition: **Google Research Football with Manchester City F.C.**

The approach is as follows, decompose the agent into specific cases (like a freekick or a 1v1) and use RL agents trained on custom scenarios for optimal decison making.

# Installs

In [1]:
%%bash
# Kaggle environments.
git clone https://github.com/Kaggle/kaggle-environments.git
cd kaggle-environments && pip install .

# GFootball environment.
apt-get update -y
apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
git clone -b v2.6 https://github.com/google-research/football.git
mkdir -p football/third_party/gfootball_engine/lib
wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so

Processing /home/alishobeiri/Desktop/Football/Notebooks/kaggle-environments
  Created wheel for kaggle-environments: filename=kaggle_environments-1.3.6-py3-none-any.whl size=99822 sha256=5fcb81f93d5470f168e8c6021b023a26310ccff538900d24e93b42d46b8b84c9
  Stored in directory: /home/alishobeiri/.cache/pip/wheels/e4/43/ce/20206e03dc66dd5b980d219f6c77c45969b63d800f28f76a9c
Successfully built kaggle-environments
  Attempting uninstall: kaggle-environments
    Found existing installation: kaggle-environments 1.3.6
    Uninstalling kaggle-environments-1.3.6:
      Successfully uninstalled kaggle-environments-1.3.6
Reading package lists...


Cloning into 'kaggle-environments'...
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Cloning into 'football'...
--2020-10-10 17:13:46--  https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.193.80, 172.217.14.240, 172.217.3.176, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.193.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45244080 (43M) [application/octet-stream]
Saving to: ‘football/third_p

# Custom Scenario

The game already has pre-built scenarios ([Repository](https://github.com/google-research/football/tree/master/gfootball/scenarios)). The Environment enables us to build custom scenarios. A scenario is a football game with specific characteristics such a limited number of players. For the scenario to be included in the environment, we must write into a file as such and build the game afterwards. The following gist shows how to create a custom scenario. 

**Select the options to customize (not all are needed) and change the variable (in capital letters), including the file name, to be the desired parameters.**

```python
%%writefile kaggle-environments/football/gfootball/scenarios/CUSTOM_NAME.py
# Imports 
from . import *

def build_scenario(builder):
    # Game Duration
    builder.config().game_duration = INTEGER
    builder.config().second_half = INTEGER
    
    # Default team decison making
    builder.config().right_team_difficulty = FLOAT #(Between 0 and 1)
    builder.config().left_team_difficulty = 1.0
    
    # ?
    builder.config().deterministic = BOOLEAN
    
    # Determing which side to be on (Optional)
    # If using this, set the teams accordingly: builder.SetTeam(first_team)
    if builder.EpisodeNumber() % 2 == 0:
        first_team = Team.e_Left
        second_team = Team.e_Right
    else:
        first_team = Team.e_Right
        second_team = Team.e_Left
    
    # Consider Offside Rule
    builder.config().offsides = BOOLEAN # Default = True
    
    # Conditions to end episodes
    builder.config().end_episode_on_score = BOOLEAN # Default = False
    builder.config().end_episode_on_out_of_play = BOOLEAN # Default = False
    builder.config().end_episode_on_possession_change = BOOLEAN # Default = False
    
    # Custom ball position
    builder.SetBallPosition(XPOS, YPOS)
    
    # Create Left Team
    builder.SetTeam(Team.e_Left)
    builder.AddPlayer(X, Y, ROLE, LAZY, CONTROLLABLE) 
    # x: x coordinate of the player in the range [-1, 1].
    # y: y coordinate of the player in the range [-0.42, 0.42].
    # role: Player's role in the game (goal keeper etc.).
    # lazy (Optional BOOLEAN default False): Computer doesn't perform any automatic actions for lazy player.
    # controllable (Optional BOOLEAN default True): Whether player can be controlled.

    # Create Right Team
    builder.SetTeam(Team.e_Right)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK,)
    builder.AddPlayer(-0.7, 0.0, e_PlayerRole_CB)
```

In [2]:
# This variable can be used for fancy animation.
# Make sure its value is the same as (or at least smaller than) the one in the scenario. 
game_duration = 300

In [3]:
%%writefile kaggle-environments/football/gfootball/scenarios/custom_scenario_1v1.py
from . import *

def build_scenario(builder):
    builder.config().game_duration = 300
    builder.config().deterministic = False
    builder.config().offsides = False
    builder.config().end_episode_on_score = True
    builder.config().end_episode_on_out_of_play = True
    builder.config().end_episode_on_possession_change = True
    builder.SetBallPosition(0.5, 0.0)
    
    builder.SetTeam(Team.e_Left)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK)
    builder.AddPlayer(0.1, -0.1, e_PlayerRole_CF)

    builder.SetTeam(Team.e_Right)
    builder.AddPlayer(-1.0, 0.0, e_PlayerRole_GK)
    builder.AddPlayer(-0.7, 0.05, e_PlayerRole_CB)

Writing kaggle-environments/football/gfootball/scenarios/custom_scenario_1v1.py


Now we can build the environment with the custom scenarios built into it.

In [4]:
%%bash
# Install
cd kaggle-environments/football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

Processing /home/alishobeiri/Desktop/Football/Notebooks/kaggle-environments/football
  Created wheel for gfootball: filename=gfootball-2.6-cp38-cp38-linux_x86_64.whl size=38671357 sha256=b9bbf03c9e8efd5e5b5f033066e36c097998d9ec2a8d173179bc7fee56289e44
  Stored in directory: /tmp/pip-ephem-wheel-cache-7lftlq08/wheels/15/5b/9b/43198a38f6a0f9b9dbda6d4956860e0f5c4dc87ebe5b9a18a9
Successfully built gfootball
  Attempting uninstall: gfootball
    Found existing installation: gfootball 2.6
    Uninstalling gfootball-2.6:
      Successfully uninstalled gfootball-2.6


In [5]:
%%bash
# Proof of Concept: verify the presence of the desired files
find / -name custom_scenario*.py | grep site-packages | cut -c -90

/home/alishobeiri/anaconda3/envs/football/lib/python3.8/site-packages/gfootball/scenarios/


find: ‘/boot/efi’: Permission denied
find: ‘/root’: Permission denied
find: ‘/home/lost+found’: Permission denied
find: ‘/run/wpa_supplicant’: Permission denied
find: ‘/run/gdm3’: Permission denied
find: ‘/run/udisks2’: Permission denied
find: ‘/run/cups/certs’: Permission denied
find: ‘/run/user/1000/inaccessible’: Permission denied
find: ‘/run/sudo’: Permission denied
find: ‘/run/speech-dispatcher’: Permission denied
find: ‘/run/openvpn-server’: Permission denied
find: ‘/run/openvpn-client’: Permission denied
find: ‘/run/systemd/resolve/netif’: Permission denied
find: ‘/run/systemd/unit-root’: Permission denied
find: ‘/run/systemd/inaccessible’: Permission denied
find: ‘/run/initramfs’: Permission denied
find: ‘/var/spool/rsyslog’: Permission denied
find: ‘/var/spool/cron/crontabs’: Permission denied
find: ‘/var/spool/cups’: Permission denied
find: ‘/var/log/private’: Permission denied
find: ‘/var/log/speech-dispatcher’: Permission denied
find: ‘/var/log/gdm3’: Permission denied
find

# Bot
[Source](https://www.kaggle.com/piotrstanczyk/gfootball-template-bot)

In [6]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *

@human_readable_agent
def agent(obs):
    # Get the position of the players to be controlled    
    # We always control left team (observations and actions
    # are mirrored appropriately by the environment).
    controlled_player_pos = obs['left_team'][obs['active']]
    
    # Offense
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        if obs["game_mode"] == 1:
            # KickOff
            return Action.ShortPass
        elif obs["game_mode"] == 2:
            # GoalKick
            return Action.LongPass
        elif obs["game_mode"] == 3:
            # FreeKick
            return Action.Shot
        elif obs["game_mode"] == 4:
            # Corner
            return Action.HighPass
        elif obs["game_mode"] == 5:
            # ThrowIn
            return Action.ShortPass
        elif obs["game_mode"] == 6:
            # Penalty
            return Action.Shot
        else:
            # Normal
            if controlled_player_pos[0] > 0.5:
                return Action.Shot
            # Run towards the goal otherwise.
            return Action.Right
    # Defense
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide
    


Writing submission.py


In [7]:
# %%writefile submission.py
# from kaggle_environments.envs.football.helpers import *
# from math import sqrt

# SPRINT_RANGE = 0.6

# SHOT_RANGE_X = 0.7  
# SHOT_RANGE_Y = 0.2

# GOALIE_OUT = 0.2
# LONG_SHOT_X = 0.4
# LONG_SHOT_Y = 0.2

# directions = [
# [Action.TopLeft, Action.Top, Action.TopRight],
# [Action.Left, Action.Idle, Action.Right],
# [Action.BottomLeft, Action.Bottom, Action.BottomRight]]

# dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)

# enemyGoal = [1, 0]
# GOALKEEPER = 0

# shot_range = [[SHOT_RANGE_X, 1], 
#                   [-SHOT_RANGE_Y, SHOT_RANGE_Y]]

# def inside(pos, area):
#     return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

# @human_readable_agent
# def agent(obs):
#     controlled_player_pos = obs['left_team'][obs['active']]
    
#     if obs["game_mode"] == GameMode.Penalty:
#         return Action.Shot
#     if obs["game_mode"] == GameMode.Corner:
#         if controlled_player_pos[0] > 0:
#             return Action.Shot
#     if obs["game_mode"] == GameMode.FreeKick:
#         return Action.Shot
    
#     # Make sure player is running down the field.
#     if  0 < controlled_player_pos[0] < SPRINT_RANGE and Action.Sprint not in obs['sticky_actions']:
#         return Action.Sprint
#     elif SPRINT_RANGE < controlled_player_pos[0] and Action.Sprint in obs['sticky_actions']:
#         return Action.ReleaseSprint

#     # If our player controls the ball:
#     if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
#         if inside(controlled_player_pos, shot_range) and controlled_player_pos[0] < obs['ball'][0]:
#             return Action.Shot
        
#         elif ( abs(obs['right_team'][GOALKEEPER][0] - 1) > GOALIE_OUT   
#                 and controlled_player_pos[0] > LONG_SHOT_X and abs(controlled_player_pos[1]) < LONG_SHOT_Y ):
#             return Action.Shot
        
#         else:
#             xdir = dirsign(enemyGoal[0] - controlled_player_pos[0])
#             ydir = dirsign(enemyGoal[1] - controlled_player_pos[1])
#             return directions[ydir][xdir]
        
#     # if we we do not have the ball:
#     else:
#         # Run towards the ball.
#         xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
#         ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
#         return directions[ydir][xdir]

In [8]:
# %%writefile submission.py
# from kaggle_environments.envs.football.helpers import *

# # @human_readable_agent wrapper modifies raw observations 
# # provided by the environment:
# # https://github.com/google-research/football/blob/master/gfootball/doc/observation.md#raw-observations
# # into a form easier to work with by humans.
# # Following modifications are applied:
# # - Action, PlayerRole and GameMode enums are introduced.
# # - 'sticky_actions' are turned into a set of active actions (Action enum)
# #    see usage example below.
# # - 'game_mode' is turned into GameMode enum.
# # - 'designated' field is removed, as it always equals to 'active'
# #    when a single player is controlled on the team.
# # - 'left_team_roles'/'right_team_roles' are turned into PlayerRole enums.
# # - Action enum is to be returned by the agent function.
# @human_readable_agent
# def agent(obs):
#     # Make sure player is running.
#     if Action.Sprint not in obs['sticky_actions']:
#         return Action.Sprint
#     # We always control left team (observations and actions
#     # are mirrored appropriately by the environment).
#     controlled_player_pos = obs['left_team'][obs['active']]
#     # Does the player we control have the ball?
#     if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
#         # Shot if we are 'close' to the goal (based on 'x' coordinate).
#         if controlled_player_pos[0] > 0.5:
#             return Action.Shot
#         # Run towards the goal otherwise.
#         return Action.Right
#     else:
#         # Run towards the ball.
#         if obs['ball'][0] > controlled_player_pos[0] + 0.05:
#             return Action.Right
#         if obs['ball'][0] < controlled_player_pos[0] - 0.05:
#             return Action.Left
#         if obs['ball'][1] > controlled_player_pos[1] + 0.05:
#             return Action.Bottom
#         if obs['ball'][1] < controlled_player_pos[1] - 0.05:
#             return Action.Top
#         # Try to take over the ball if close to the ball.
#         return Action.Slide

# Environment Setup

Now we can initialize the environment with the desired scenario (ours or the pre-built ones) and use our agent on it.

In [9]:
from kaggle_environments import make

scenarios = {0: "11_vs_11_competition",
             1: "11_vs_11_easy_stochastic",
             2: "11_vs_11_hard_stochastic",
             3: "11_vs_11_kaggle",
             4: "11_vs_11_stochastic",
             5: "1_vs_1_easy",
             6: "5_vs_5",
             7: "academy_3_vs_1_with_keeper",
             8: "academy_corner",
             9: "academy_counterattack_easy",
             10: "academy_counterattack_hard",
             11: "academy_empty_goal",
             12: "academy_empty_goal_close",
             13: "academy_pass_and_shoot_with_keeper",
             14: "academy_run_pass_and_shoot_with_keeper",
             15: "academy_run_to_score",
             16: "academy_run_to_score_with_keeper",
             17: "academy_single_goal_versus_lazy",
             18: "custom_scenario_1v1"
             }

# Choose here the scenario of interest
env = make("football", debug=True,
           configuration={"save_video": True, 
                          "scenario_name": scenarios[0], 
                          "running_in_notebook": True})
         
# Choose here the agents that play
output = env.run(["/kaggle/working/submission.py", "do_nothing"])
scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))

print('Left player: reward = %s, status = %s, info = %s' % (output[-1][0]['reward'], output[-1][0]['status'], output[-1][0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[-1][1]['reward'], output[-1][1]['status'], output[-1][1]['info']))

# Optional default Game Rendering
env.render(mode="human", width=600, height=400)

ModuleNotFoundError: No module named 'kaggle_environments'

# Fancy Animation
[Source](https://www.kaggle.com/jaronmichal/human-readable-visualization)

In [ ]:
import matplotlib.patches as patches
from  matplotlib.patches import Arc
from matplotlib import pyplot as plt
from matplotlib import animation
import matplotlib.patches as mpatches

from enum import Enum

import math

import numpy as np
from IPython.display import HTML

# Change size of figure
plt.rcParams['figure.figsize'] = [20, 16]
# Function that draw the green pitch with white lines
def drawPitch(width, height, color="w"):    
    fig = plt.figure()
    ax = plt.axes(xlim=(-10, width + 10), ylim=(-15, height + 5))
    plt.axis('off')

    # Grass around pitch
    rect = patches.Rectangle((-5,-5), width + 10, height + 10, linewidth=1, edgecolor='gray',facecolor='#3f995b', capstyle='round')
    ax.add_patch(rect)

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), width, height, ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    # Middle line
    plt.plot([width/2, width/2], [0, height], color=color, linewidth=2)

    # Dots
    dots_x = [11, width/2, width-11]
    for x in dots_x:
        plt.plot(x, height/2, 'o', color=color, linewidth=2)

    # Penalty box  
    penalty_box_dim = [16.5, 40.3]
    penalty_box_pos_y = (height - penalty_box_dim[1]) / 2

    rect = plt.Rectangle((0, penalty_box_pos_y), penalty_box_dim[0], penalty_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, penalty_box_pos_y), -penalty_box_dim[0], penalty_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    #Goal box
    goal_box_dim = [5.5, penalty_box_dim[1] - 11 * 2]
    goal_box_pos_y = (penalty_box_pos_y + 11)

    rect = plt.Rectangle((0, goal_box_pos_y), goal_box_dim[0], goal_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, goal_box_pos_y), -goal_box_dim[0], goal_box_dim[1], ec=color, fc="None", lw=2)
    ax.add_patch(rect)

    #Goals
    rect = plt.Rectangle((0, penalty_box_pos_y + 16.5), -3, 7.5, ec=color, fc=color, lw=2, alpha=0.3)
    ax.add_patch(rect)
    rect = plt.Rectangle((width, penalty_box_pos_y + 16.5), 3, 7.5, ec=color, fc=color, lw=2, alpha=0.3)
    ax.add_patch(rect)

    # Middle circle
    mid_circle = plt.Circle([width/2, height/2], 9.15, color=color, fc="None", lw=2)
    ax.add_artist(mid_circle)


    # Penalty box arcs
    left  = patches.Arc([11, height/2], 2*9.15, 2*9.15, color=color, fc="None", lw=2, angle=0, theta1=308, theta2=52)
    ax.add_patch(left)
    right = patches.Arc([width - 11, height/2], 2*9.15, 2*9.15, color=color, fc="None", lw=2, angle=180, theta1=308, theta2=52)
    ax.add_patch(right)

    # Arcs on corners
    corners = [[0, 0], [width, 0], [width, height], [0, height]]
    angle = 0
    for x,y in corners:
        c = patches.Arc([x, y], 2, 2, color=color, fc="None", lw=2, angle=angle,theta1=0, theta2=90)
        ax.add_patch(c)
        angle += 90
    return fig, ax

WIDTH = 105
HEIGHT = 68
X_RESIZE = WIDTH
Y_RESIZE = HEIGHT / 0.42

class GameMode(Enum):
    Normal = 0
    KickOff = 1
    GoalKick = 2
    FreeKick = 3
    Corner = 4
    ThrowIn = 5
    Penalty = 6

def scale_x(x):
    return (x + 1) * (X_RESIZE/2)

def scale_y(y):
    return (y + 0.42) * (Y_RESIZE/2)

def extract_data(frame):
    # Function that gets game data at each time frame
    res = {}
    obs = frame[0]['observation']['players_raw'][0]
    res["left_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["left_team"]]
    res["right_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["right_team"]]

    ball_x, ball_y, ball_z = obs["ball"]
    res["ball"] = [scale_x(ball_x),  scale_y(ball_y), ball_z]
    res["score"] = obs["score"]
    res["steps_left"] = obs["steps_left"]
    res["ball_owned_team"] = obs["ball_owned_team"]
    res["ball_owned_player"] = obs["ball_owned_player"]
    res["right_team_roles"] = obs["right_team_roles"]
    res["left_team_roles"] = obs["left_team_roles"]
    res["left_team_direction"] = obs["left_team_direction"]
    res["right_team_direction"] = obs["right_team_direction"]
    res["game_mode"] = GameMode(obs["game_mode"]).name
    return res

def draw_team(obs, team, side):
    X = []
    Y = []
    for x, y in obs[side]:
        X.append(x)
        Y.append(y)
    team.set_data(X, Y)

def draw_ball(obs, ball):
    # Function that draws the ball on the field    
    ball.set_markersize(10 + 5 * obs["ball"][2]) # Scale size of ball based on height
    ball.set_data(obs["ball"][:2])

def draw_ball_owner(obs, ball_owner, team_active):
    # Function that highlights the player in possession of the ball    
    if obs["ball_owned_team"] == 0:
        x, y = obs["left_team"][obs["ball_owned_player"]]
        ball_owner.set_data(x, y)
        team_active.set_data(WIDTH / 4 + 7, -7)
        team_active.set_markerfacecolor("red")
    elif obs["ball_owned_team"] == 1:
        x, y = obs["right_team"][obs["ball_owned_player"]]
        ball_owner.set_data(x, y)
        team_active.set_data(WIDTH / 4 + 50, -7)
        team_active.set_markerfacecolor("blue")
    else:
        ball_owner.set_data([], [])
        team_active.set_data([], [])
    
def draw_players_directions(obs, directions, side):
    # Function that direction in which players run
    index = 0
    if "right" in side:
        index = 11
    for i, player_dir in enumerate(obs[f"{side}_direction"]):
        x_dir, y_dir = player_dir
        dist = math.sqrt(x_dir ** 2 + y_dir ** 2) + 0.00001 # to prevent division by 0
        x = obs[side][i][0]
        y = obs[side][i][1] 
        directions[i + index].set_data([x, x + x_dir / dist ], [y, y + y_dir / dist])

# Settup for the animation
fig, ax = drawPitch(WIDTH, HEIGHT)
ax.invert_yaxis()

ball_owner, = ax.plot([], [], 'o', markersize=30,  markerfacecolor="yellow", alpha=0.5)
team_active, = ax.plot([], [], 'o', markersize=30,  markerfacecolor="blue", markeredgecolor="None")

team_left, = ax.plot([], [], 'o', markersize=20, markerfacecolor="r", markeredgewidth=2, markeredgecolor="white")
team_right, = ax.plot([], [], 'o', markersize=20,  markerfacecolor="b", markeredgewidth=2, markeredgecolor="white")

ball, = ax.plot([], [], 'o', markersize=10,  markerfacecolor="black", markeredgewidth=2, markeredgecolor="white")
text_frame = ax.text(-5, -5, '', fontsize=25)
match_info = ax.text(105 / 4 + 10, -5, '', fontsize=25)
game_mode = ax.text(105 - 25, -5, '', fontsize=25)
goal_notification = ax.text(105 / 4 + 10, 0, '', fontsize=25)

# Drawing of directions definitely can be done in a better way
directions = []
for i in range(22):
    direction, = ax.plot([], [], color='yellow', lw=3)
    directions.append(direction)

drawings = [team_active, ball_owner, team_left, team_right, ball, text_frame, match_info, game_mode, goal_notification]

def init():
    team_left.set_data([], [])
    team_right.set_data([], [])
    ball_owner.set_data([], [])
    team_active.set_data([], [])
    ball.set_data([], [])
    return drawings 

def animate(i):
    global prev_score_a, prev_score_b
    obs = extract_data(output[i])

    # Draw info about ball possesion
    draw_ball_owner(obs, ball_owner, team_active)

    # Draw players
    draw_team(obs, team_left, "left_team")
    draw_team(obs, team_right, "right_team")

    draw_players_directions(obs, directions, "left_team")
    draw_players_directions(obs, directions, "right_team")

    draw_ball(obs, ball)

    # Draw textual informations
    text_frame.set_text(f"Frame: {i}/{obs['steps_left'] + i - 1}")
    game_mode.set_text(f"Game mode: {obs['game_mode']}")

    score_a, score_b = obs["score"]
    match_info.set_text(f"Left team {score_a} : {score_b} Right Team")

    return drawings  

In [ ]:
# Build the animation and draw it
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=game_duration, interval=100, blit=True)

HTML(anim.to_html5_video())